<a href="https://colab.research.google.com/github/shahaansshah/3253-083_Group9/blob/main/3253_Term_Project_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
import json
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import VotingRegressor


class HousingDataProcessor:
    """
    Class to process and prepare housing data for machine learning models.
    """

    def __init__(self, file_path):
        """
        Initializes the HousingDataProcessor with the path to the JSON data file.

        Parameters:
        -----------
        file_path : str
            The path to the JSON data file.
        """
        self.file_path = file_path
        self.df = self.load_and_process_data()

    def load_and_process_data(self):
        """
        Loads the JSON data, flattens the nested structure, and performs initial data cleaning.

        Returns:
        --------
        pandas.DataFrame
            The processed DataFrame.
        """
        records = []
        with open(self.file_path, 'r') as f:
            for line in f:
                try:
                    data = json.loads(line)
                    record = data.get('data')
                    if record is not None:
                        features = {}
                        for feature_group in record.get('features', []):
                            for feature_category in feature_group.get('value', []):
                                for feature in feature_category.get('value', []):
                                    features[f"{feature_group['name']}_{feature_category['name']}_{feature['name']}"] = feature['value']

                        record.update(features)
                        record.pop('features', None)
                        records.append(record)
                except json.JSONDecodeError as e:
                    print(f"Skipping invalid JSON line: {line}")

        df = pd.DataFrame(records)
        return df

    def clean_data(self):
        """
        Removes irrelevant columns, handles missing values, and prepares categorical features.
        """

        # Inspect columns with missing data
        self.inspect_missing_data()

        # Clean up column names and keep only relevant columns
        self.rationalize_columns()

        # Normalize the values of the amenities columns
        self.normalize_amenities()

        # Calculate distance to lake
        self.calculate_distance_to_lake()

        # Extract square footage from description
        self.extract_square_footage_from_description()

        # Drop propertyType in 'FARM', 'Land', 'Other', 'Multi Family'
        self.df = self.df[~self.df['propertyType'].isin(['FARM', 'Land', 'Other', 'Multi_Family'])]

        # One-hot encode categorical features
        self.one_hot_encode_categorical_features()

        # Handle missing values (similar to your original code, but using methods)
        self.handle_missing_values()


    def inspect_missing_data(self):
      # look up which fields contain a lot of missing data
      missing_values = [col for col in self.df.columns if self.df[col].isnull().any()]
      print('Number of columns with missing values:', len(missing_values), '\n')

      for col in missing_values:
        print(col, round(self.df[col].isnull().mean(), 3)*100, '% missing values')

      print('\n***************\n\n')


    def rationalize_columns(self):
        # Clean up column names
        self.df.columns = self.df.columns.str.replace(r' +', '_', regex=True)

        # Keep only relevant columns
        columns_to_keep = ['latitude', 'listPrice', 'longitude', 'lotSize', 'numBathrooms',
       'numBedrooms', 'parking', 'propertyType', 'yearBuilt', 'garage',
       'Amenities_Utilities_Heating_Cooling_Cooling',
       'Amenities_Utilities_Heating_Cooling_Heat_Source']
        self.df = self.df[columns_to_keep]


    def normalize_amenities(self):
        """
        Categorical variables levels cleanup
        """
        self.df['Amenities_Utilities_Heating_Cooling_Cooling'] = self.df['Amenities_Utilities_Heating_Cooling_Cooling'].str.replace(r'Ductless.*|.*Central .*', 'Y', regex=True)
        self.df['Amenities_Utilities_Heating_Cooling_Cooling'] = self.df['Amenities_Utilities_Heating_Cooling_Cooling'].str.replace(r'^(?!.*Y).*', 'N', regex=True)
        self.df['Amenities_Utilities_Heating_Cooling_Heat_Source'] = self.df['Amenities_Utilities_Heating_Cooling_Heat_Source'].str.replace(r'Electric.*', 'Electric', regex=True)
        self.df['Amenities_Utilities_Heating_Cooling_Heat_Source'] = self.df['Amenities_Utilities_Heating_Cooling_Heat_Source'].str.replace(r'Natural gas.*', 'Gas', regex=True)
        self.df['Amenities_Utilities_Heating_Cooling_Heat_Source'] = self.df['Amenities_Utilities_Heating_Cooling_Heat_Source'].str.replace(r'^(?!Electric|Gas).*', 'Other', regex=True)


    def calculate_distance_to_lake(self):
        """
        Calculates the minimum distance to lake points for each property.
        """
        def distance(origin, destination):
            """
            Calculate the Haversine distance.

            Parameters:
            ----------
            origin : tuple of float
                (latitude, longitude)
            destination : tuple of float
                (latitude, longitude)

            Returns:
            -------
            distance_in_km : float
            """
            lat1, lon1 = origin
            lat2, lon2 = destination
            radius = 6371  # km

            dlat = math.radians(lat2 - lat1)
            dlon = math.radians(lon2 - lon1)
            a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
                 math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
                 math.sin(dlon / 2) * math.sin(dlon / 2))
            c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
            d = radius * c

            return d

        shorepoints = [(43.337573, -79.769493), (43.325047, -79.792023), (43.346755, -79.758282), (43.352226, -79.751185),
              (43.362459, -79.737418), (43.366887, -79.729100), (43.385790, -79.712277), (43.399670, -79.700950),
              (43.419369, -79.683863), (43.451628, -79.654371), (43.467697, -79.640033), (43.486686, -79.617320),
              (43.517525, -79.601415), (43.538370, -79.594686), (43.562755, -79.564711), (43.576052, -79.543607),
              (43.594663, -79.503232), (43.625224, -79.478151), (43.630317, -79.433801), (43.632973, -79.407191),
              (43.272289, -79.919314), (43.276725, -79.860298), (43.271462, -79.833200), (43.251221, -79.757855),
          ]

        distances = []
        for id, row in self.df.iterrows():
            mindist = distance((row["latitude"], row["longitude"]), shorepoints[0])
            for point in shorepoints:
                dist = distance((row["latitude"], row["longitude"]), point)
                if dist < mindist:
                    mindist = dist
            distances.append(mindist)
        self.df.insert(len(self.df.iloc[0]), "distance_to_lake", distances)


    def extract_square_footage_from_description(self):
      """
      Extracts square footage from the 'description' column.
      """
      """
      ## THIS CODE IS ONLY PRESENT TO SHOW HOW DATA WAS COLLECTED AND DOES NOT HAVE THE REQUIRED API KEY TO RUN
      ## Completing the dataset by adding missing entries of square footage using the description which includes square footage (usually) and using the AzureAI chatbot to analyze text and return the square footage or None.

      # Connection to openai chatbot
      deployment_name = "REDACTED"
      client = AzureOpenAI(
          api_key= "REDACTED", ## I am not allowed to share this API key for legal reasons but it was used to message the Azure AI chatbot
          api_version="2024-02-01",
          azure_endpoint = "REDACTED"
          )

      def message_chatbot(message, description):

          # Uses the connecetion made to make a request to the Azure openai chatbot
          response = client.chat.completions.create(
              model=deployment_name,
              temperature=0.7,
              max_tokens=400,

              # Both messages to be submitted
              messages=[
                  {"role": "system", "content": message},
                  {"role": "user", "content": description}
              ]
          )
          generated_text = response.choices[0].message.content

          return (generated_text)

      message = "The following will be a description of a house, Please find the total square footage of the house and return it as a single number without any commas or measurements. Do not return any other text other than the number itself, without any math equations or explanations, just a number. If there is no square footage found return 'None'"
      sqfootage = []
      for id, row in df_cleaned.iterrows():
          if row["sqftTotalRaw"] == 0 or row["sqftTotalRaw"] == None:
              sqfootage.append(message_chatbot(message, row["description"]))
          else:
              sqfootage.append(row["sqftTotalRaw"])
      """
      ## Because the above code does not work without the API (and I am not allowed to put it here), this is what it wouldve generated, sorry for the hardcoding :( to be fair this isn't part of the assignment and was extra work
      ## HARDCODING PRESENT DUE TO NOT HAVING THE API KEY
      sqfootage = [
      '1200',
      'None', 'None', '600', '3993', '2816', 'None', '1350', '3180', '1075', '1824', '2980', 'None', '3262', '1000', '1191', '4487', 'None', 'None', '1108', 'None',
      '2900', '5186', 'None', '1479', '4000', '3100', '950', 'None', '2500', '5246', 'None', 'None', '3214', '1400', 'None', '2000', '3537', 'None', '1800', '3284',
      '1762', '4303', '4100', '1936', '1017', '2617', '9900', '1000', '79954', '613', '1950', 'None', '2500', 'None', '1800', '1088', '1911', '2159', '1700', '4367',
      '5293', '1200', '1918', '986', '717', '2000', '1146', '6000', 'None', 'None', 'None', 'None', '3860', '6984', 'None', '3800', '715', 'None', '1990', '7476',
      'None', '610', 'None', 'None', '711', '1351', '1617', 'None', 'None', 'None', 'None', '2000', 'None', 'None', 'None', 'None', '1161', 'None', '5000', '2595',
      'None', '2702', 'None', 'None', 'None', '2018', '660', 'None', '6568', 'None', 'None', 'None', '1700', 'None', '3300', '3108', '2720', 'None', 'None', 'None',
      'None', 'None', 'None', 'None', '1075', '2805', '4600', '3614', '2500', 'None', '1710', '1285', '4655', 'None', '2944', '4038', 'None', 'None', '3285', 'None',
      '3800', '2150', 'None', '1900', 'None', 'None', '2000', 'None', '1535', '3124', 'None', '845', 'None', '3234', 'None', '1400', 'None', 'None', 'None', '995',
      '3100', 'None', '3873', 'None', '3886', '3400', '1783', 'None', 'None', '1900', 'None', '1557', '1783', '2918', '1300', 'None', '946', '592', 'None', 'None',
      'None', '715', 'None', '3680', '3610', 'None', 'None', '2857', '1850', '3262', 'None', 'None', '2440', '2077', 'None', '4000', '6127', '479', 'None', '2734',
      '2791', 'None', 'None', '1462', 'None', 'None', 'None', '840', 'None', 'None', '6772', 'None', 'None', '4401', '3550', 'None', '756', 'None', '4423', '2250',
      'None', 'None', 'None', '3718', 'None', '9907', 'None', 'None', 'None', '1250', 'None', 'None', 'None', 'None', 'None', '3081', '2953', 'None', '962', '1535',
      'None', 'None', '1700', '6400', 'None', '1250', '7494', 'None', '500', 'None', '2159', 'None', 'None', 'None', 'None', '528', '2150', '2406', 'None', 'None',
      'None', '3143', 'None', '2856', '1900', '2789', 'None', '2767', '3000', '4364', 'None', '3921', '4000', '4680', 'None', '6416', '3430', '8750', '1000', '7500',
      '922', '1790', '2981', '1351', 'None', 'None', 'None', 'None', '3500', 'None', '4200', '3500', '2326', 'None', '3000', 'None', '1288', 'None', '1800', '1774',
      'None', '2000', '2150', '1170', '1408', '1', 'None', '5000', 'None', '1050', 'None', '2700', '2720', '1000', '3200', 'None', 'None', '2820', 'None', 'None',
      'None', '3200', '630', '3550', '1548', 'None', '1084', '8000', '1455', 'None', '6568', 'None', '5470', '2500', 'None', '3368', '820', 'None', 'None', '2041',
      '735', '1762', '4303', '854', 'None', '3400', 'None', '855', 'None', 'None', 'None', 'None', 'None', 'None', '2946', '1079', '4400', '1408', 'None', '6424',
      'None', 'None', 'None', '1990', 'None', 'None', '2210', '2593', '735', 'None', 'None', '4600', '730', '5922', '2000', 'None', '5883', 'None', '3000', '7400',
      'None', '1943', '997', '1783', '1780', '2247', 'None', '3716', '2243', '6470', '854', '1413', 'None', 'None', 'None', '6220', 'None', '1411', '1428', '1258',
      '1780', 'None', '1954', 'None', '700', '1400', '1200', '655', '2200', 'None', 'None', 'None', '2744', '3700', '1880', 'None', '6829', '1050', '1615', 'None',
      'None', 'None', 'None', '774', '1577', 'None', 'None', 'None', 'None', '700', 'None', '5000', '714', '2300', '6100', 'None', 'None', '1264', '1119', 'None',
      '5745', '4279', 'None', '3558', 'None', '4728', 'None', '1321', '4364', 'None', 'None', 'None', 'None', '2691', '1830', '4000', 'None', '972', '980', '3019',
      'None', 'None', 'None', '7715', '586', '4482', '900', 'None', '3800', '4000', '3600', 'None', '4867', '1113', '943', '620', '1250', 'None', 'None', '2971',
      '2816', '1200', '1630', 'None', 'None', 'None', '2900', 'None', '625', 'None', 'None', '3173', 'None', 'None', '3815', 'None', '2056', '1000', 'None', '3815',
      'None', '3316', '5000', '4000', '972', 'None', 'None', 'None', '1948', '4200', 'None', 'None', 'None', '2065', 'None', 'None', '3653', '1188', '2321', 'None',
      '1201', 'None', '5700', 'None', '4054', '2728', '2807', '2704', 'None', '1866', 'None', '1700', '1501', 'None', '3325', '3800', '2459', '933', '2026', '629',
      'None', 'None', '2790', '4200', '3180', '650', '2479', '613', '3700', 'None', '1830', 'None', '3999', 'None', 'None', 'None', 'None', 'None', '3605', 'None',
      'None', '4750', '1700', '1450', '3000', '916', 'None', '4597', 'None', 'None', 'None', 'None', 'None', 'None', '2000', 'None', 'None', 'None', '13400', '1700',
      'None', '469', 'None', '9413', 'None', '3741', '830', '1290', 'None', 'None', 'None', '2677', 'None', 'None', 'None', 'None', 'None', '13400', 'None', 'None',
      '5400', '2500', 'None', 'None', 'None', 'None', 'None', '2064', '3447', 'None', '830', 'None', '4000', 'None', '839', 'None', 'None', '3880', '1586', 'None',
      '1650', 'None', 'None', 'None', 'None', '1800', 'None', 'None', 'None', '1100', '1300', 'None', 'None', 'None', 'None', 'None', '2768', 'None', 'None', 'None',
      '1500', '1600', 'None', 'None', 'None', '827', 'None', 'None', 'None', 'None', 'None', 'None', '1480', 'None', 'None', 'None', 'None', '1186', '7700', '1400',
      'None', 'None', '640', '988', 'None', 'None', 'None', '576', 'None', 'None', 'None', 'None', '1000', 'None', '1655', 'None', 'None', 'None', 'None', 'None',
      '4000', 'None', 'None', '600', '3000', '9334', 'None', '1000', '1300', 'None', 'None', '2676', '1300', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
      'None', '1800', 'None', '784', '590', 'None', '3111', '608', 'None', 'None', 'None', 'None', 'None', '1405', '841', '1087', 'None', 'None', 'None', '1136',
      'None', '1100', '1870', 'None', 'None', '2777', 'None', 'None', 'None', 'None', 'None', 'None', '400', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
      'None', '3000', 'None', 'None', 'None', '756', '821', '1347', 'None', 'None', '1150', 'None', 'None', '5500', 'None', 'None', '910', '1300', 'None', 'None',
      '1655', 'None', 'None', '1350', '1575', '566', 'None', 'None', '800', 'None', '1100', '948', '1100', '1203', '1186', 'None', 'None', 'None', '1032', 'None',
      'None', '9334', '4410', '1179', '965', 'None', 'None', 'None', 'None', 'None', '965', 'None', 'None', 'None', 'None', '2651', 'None', 'None', 'None', 'None',
      '1275', 'None', 'None', 'None', 'None', 'None', 'None', '560', '691', 'None', '3500', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
      'None', '4300', 'None', 'None', '600', 'None', '1500', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '3000', 'None', 'None', 'None', 'None', 'None',
      'None', 'None', '2924', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '2500', '723', 'None', '4126', '630', 'None', 'None', 'None', 'None', '800',
      'None', 'None', 'None', '821', 'None', 'None', '562', 'None', 'None', 'None', '1685', '846', '3000', 'None', '1933', 'None', 'None', 'None', 'None', '5000',
      '4400', 'None', 'None', 'None', 'None', '1100', '4760', '2986', '836', 'None', 'None', 'None', 'None', 'None', 'None', '560', '3638', '1136', 'None', 'None',
      'None', 'None', '4000', 'None', '910', 'None', '982', '3282', 'None', 'None', '700', 'None', 'None', 'None', 'None', 'None', '3063', '3631', '2300', 'None',
      'None', 'None', '723', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '1050', '576', 'None', 'None', '940', 'None', 'None',
      'None', 'None', 'None', 'None', '1370', 'None', '5500', '2636', 'None', '1138', 'None', '800', 'None', 'None', '1138', 'None', 'None', 'None', '716', 'None',
      'None', 'None', 'None', '3705', '1360', '3296', 'None', 'None', 'None', 'None', 'None', 'None', '540', 'None', '1700', 'None', 'None', 'None', 'None', '562',
      '953', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '1500', 'None', 'None', '703', '4000', 'None', '2360', 'None', 'None',
      '2020', '1347', 'None', 'None', 'None', '1084', 'None', '1100', 'None', 'None', 'None', 'None', '580', '1174', 'None', 'None', '982', 'None', 'None', '728',
      'None', '810', '795', 'None', 'None', '810', 'None', 'None', '1690', 'None', 'None', '1625', 'None', '6500', '2500', 'None', 'None', '784', 'None', '3073',
      'None', 'None', 'None', 'None', '756', '1032', '2000', 'None', 'None', 'None', 'None', 'None', '2300', 'None', '700', 'None', '2215', '1179', 'None', 'None',
      'None', '630', '2300', '1290', '1000', '3000', '799', 'None', 'None', 'None', '2064', 'None', 'None', '4400', 'None', 'None', '1343', 'None', 'None', 'None',
      'None', 'None', 'None', '1150', '562', '566', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '1200', '875', 'None', 'None', '948', 'None', 'None',
      'None', 'None', 'None', 'None', 'None', 'None', 'None', '1041', '1655', 'None', 'None', 'None', 'None', 'None', '1300', 'None', 'None', 'None', 'None', 'None',
      '2070', '562', 'None', 'None', 'None', '5000', 'None', 'None', '515', 'None', '2050', '2100', 'None', 'None', 'None', 'None', '1626', 'None', 'None', 'None',
      'None', 'None', '846', 'None', 'None', 'None', '5000', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
      '3500', '4075', 'None', 'None', 'None', '1025', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '800', 'None', 'None', 'None', '563', '2000', '1483',
      'None', 'None', 'None', 'None', '1405', 'None', 'None', 'None', '1100', 'None', '2197', 'None', 'None', '2677', 'None', '1338', 'None', '1897', 'None', 'None',
      '1405', '2200', 'None', '830', 'None', '1483', 'None', 'None', 'None', '1157', 'None', 'None', '1369', 'None', 'None', '2622', '1000', '1010', 'None', '2194',
      'None', 'None', '4000', 'None', 'None', '1549', 'None', '3000', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
      'None', 'None', 'None', 'None', '2100', 'None', 'None', 'None', 'None', 'None', '993', 'None', '785', '2309', 'None', '1132', '1447', 'None', 'None', 'None',
      'None', 'None', 'None', '1416', 'None', 'None', 'None', '2600', 'None', 'None', 'None', '984', 'None', 'None', 'None', 'None', 'None', '1383', 'None', 'None',
      'None', 'None', '1141', 'None', '0000', '3297', '1358', 'None', '3000', '3092', 'None', 'None', 'None', 'None', 'None', 'None', '1860', '1158', '1772', 'None',
      'None', 'None', 'None', 'None', 'None', '1500', '1483', 'None', 'None', 'None', 'None', 'None', 'None', '1900', '1313', 'None', 'None', 'None', 'None', 'None',
      '1300', 'None', '1441', '3092', '1000', '2500', 'None', 'None', '2350', '2651', 'None', 'None', 'None', 'None', '1589', 'None', '1736', 'None', 'None', 'None',
      '2812', '1324', 'None', 'None', '4409', '1072', 'None', 'None', 'None', 'None', '3400', '2700', '2158', 'None', '1185', '2252', 'None', 'None', 'None', '1615',
      'None', '879', '2615', 'None', '1978', 'None', '1000', '998', 'None', 'None', 'None', '1897', 'None', '1608', 'None', '1500', 'None', '700', 'None', 'None',
      'None', 'None', '846', '1975', 'None', '2900', 'None', 'None', 'None', 'None', '2200', 'None', 'None', 'None', '2000', 'None', '2986', 'None', 'None', 'None',
      'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '1050', 'None', 'None', 'None', 'None', 'None', '1493', '2069', 'None', 'None', 'None', 'None',
      '2821', 'None', 'None', 'None', '2000', '1657', 'None', 'None', 'None', 'None', 'None', 'None', '3619', '2472', '2525', 'None', 'None', 'None', '1500', 'None',
      '1550', 'None', 'None', 'None', '2068', 'None', 'None', 'None', '633', 'None', '1758', 'None', '1897', 'None', '2000', 'None', 'None', 'None', '780', 'None',
      'None', 'None', '1500', 'None', 'None', 'None', '2546', 'None', 'None', 'None', 'None', 'None', '1975', 'None', 'None', 'None', 'None', '4275', 'None', '2000',
      '2615', 'None', 'None', 'None', 'None', '670', '1385', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '5000', 'None', '780', 'None', 'None', 'None',
      'None', 'None', 'None', 'None', 'None', '605', 'None', 'None', '565', 'None', 'None', 'None', 'None', '1660', 'None', '1369', '1400', 'None', 'None', 'None',
      '700', '2000', 'None', '3368', 'None', '3500', 'None', '1634', '1141', '2843', 'None', '1041', 'None', '2070', 'None', 'None', 'None', 'None', 'None', '2036',
      'None', '5000', 'None', 'None', 'None', '2100', 'None', 'None', '3740', 'None', '1319', '846', 'None', 'None', 'None', '3000', 'None', 'None', 'None', 'None',
      'None', '2500', 'None', '1900', '2252', 'None', 'None', 'None', 'None', 'None', '1582', '2240', '1897', '1235', 'None', '563', 'None', '1200', '3000', 'None',
      'None', 'None', 'None', '1290', 'None', 'None', 'None', 'None', '650', 'None', '2500', '2300', '2800', 'None', 'None', 'None', 'None', 'None', '3000', 'None',
      'None', 'None', '3000', '1453', 'None', '1800', 'None', 'None', '1100', '2400', '645', 'None', 'None', 'None', '2000', 'None', 'None', 'None', 'None', 'None',
      '900', '1002', 'None', 'None', 'None', '1390', 'None', 'None', '1497', 'None', 'None', '1000', 'None', '1240', 'None', '789', 'None', 'None', 'None', '3000',
      'None', 'None', 'None', 'None', '2688', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '1259', 'None', 'None',
      '3000', '885', '1100', 'None', 'None', 'None', 'None', 'None', 'None', '815', 'None', 'None', '756', 'None', '1150', 'None', 'None', '600', 'None', '1655',
      'None', 'None', '3000', 'None', 'None', '3000', 'None', 'None', 'None', 'None', '800', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '9334',
      'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '641', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
      'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '641', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '1967',
      'None', 'None', 'None', 'None', 'None', 'None', '5000', 'None', 'None', 'None', '2064', 'None', '1522', '800', 'None', 'None', 'None', 'None', 'None', '1259',
      'None', 'None', 'None', 'None', 'None', '1951', 'None', 'None', '1100', 'None', 'None', '4750', '1002', 'None', 'None', 'None', 'None', 'None', 'None', '3700',
      '4000', 'None', 'None', 'None', 'None', 'None', 'None', '1014', '885', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '765', 'None', 'None',
      'None', 'None', 'None', '1600', 'None', 'None', 'None', 'None', 'None', '2500', 'None', 'None', 'None', 'None', '860', 'None', 'None', 'None', 'None', 'None',
      'None', '5000', 'None', '800', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '562', '953', 'None', 'None', 'None', '1200', 'None', 'None', 'None',
      'None', '980', 'None', 'None', 'None', 'None', 'None', '1347', 'None', 'None', 'None', 'None', '1100', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
      '443', 'None', '695', 'None', 'None', 'None', 'None', 'None', 'None', '1690', '6500', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '695', 'None',
      'None', 'None', '443', '13577', 'None', '2300', '1290', 'None', 'None', 'None', '2400', 'None', '8000', 'None', '1400', 'None', '1240', 'None', 'None', 'None',
      '1063', 'None', 'None', 'None', 'None', 'None', 'None', '562', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
      'None', '600', '4000', 'None', 'None', 'None', 'None', 'None', '2064', '4200', '3400', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
      'None', 'None', 'None', 'None', '1330', '1478', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '1200', 'None', 'None', 'None',
      'None', 'None', 'None', 'None', 'None', '2581', '720', '1950', 'None', 'None', '1214', 'None', 'None', 'None', '4280', '3500', 'None', '4000', 'None', 'None',
      '4750', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '2790', 'None', '1060', 'None', 'None', 'None', 'None',
      'None', '2167', '3000', 'None', 'None', 'None', 'None', 'None', '1950', 'None', 'None', '1100', 'None', 'None', 'None', 'None', '1045', 'None', 'None', 'None',
      'None', 'None', 'None', 'None', 'None', '700', 'None', 'None', 'None', 'None', '859', 'None', '4000', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
      '1060', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
      'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '2525', 'None', 'None', 'None', 'None', 'None', 'None', '1804', 'None',
      '2600', 'None', 'None', 'None', 'None', '3000', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '1290', 'None',
      'None', 'None', '5000', '2921', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '4541', '5000', 'None', '672', '1599', '3000', '4000',
      'None', 'None', 'None', '1082', 'None', 'None', 'None', '800', 'None', 'None', 'None', 'None', 'None', '4000', 'None', 'None', 'None', '7166', 'None', 'None',
      'None', 'None', 'None', 'None', 'None', '1043', 'None', '2000', 'None', 'None', 'None', '1585', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
      'None', '6000', 'None', 'None', 'None', '1682', 'None', 'None', 'None', '7000', 'None', '1214', 'None', 'None', 'None', '1600', '4000', 'None', 'None', '1060',
      'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '1100', 'None', 'None', 'None', '5400', 'None', 'None',
      '2600', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '1430', '5000', 'None', 'None', 'None', '2000', 'None', 'None', 'None', 'None', 'None',
      'None', 'None', '4570', '854', '1024', 'None', '1599', 'None', 'None', 'None', '4500', 'None', '2500', 'None', '1800', 'None', 'None', 'None', 'None', '14863',
      'None', 'None', 'None', 'None', 'None', 'None', '2600', 'None', 'None', '1261', 'None', '4000', 'None', '3074', 'None', 'None', 'None', 'None', 'None', 'None',
      'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '4000', '966', '620', '1000', 'None', 'None', '2800', '3333',
      'None', 'None', 'None', 'None', 'None', 'None', 'None', '765', 'None', 'None', 'None', '2644', 'None', 'None', 'None', '1024', 'None', 'None', 'None', 'None',
      'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '3448', '1325', 'None', 'None', 'None', 'None', '1851', 'None', 'None', '1127',
      'None', 'None', '1095', '827', '5000', 'None', '558', 'None', 'None', 'None', '1339', '4000', 'None', 'None', '1300', 'None', '1300', 'None', '1097', 'None',
      'None', 'None', 'None', 'None', 'None', 'None', '827', 'None', 'None', '995', 'None', '4000', '641', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
      '720', 'None', '1800', 'None', 'None', 'None', 'None', 'None', 'None', '3800', 'None', '664', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
      'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
      '943', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '966', 'None', '1049', '1049', '1075', 'None', 'None', 'None', 'None', 'None', 'None', '713',
      '1100', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '600', 'None', 'None', 'None', 'None', '1095', 'None', '760', 'None', 'None', '1043', 'None',
      'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '2650', 'None', '672', 'None', '2288', 'None', 'None', 'None', 'None', 'None',
      'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', '4000', 'None', '8400', '2000', 'None',
      '2184', 'None', 'None', '2950', '1127', 'None', 'None', 'None', 'None', '1100', '1900', 'None', '1733', '1600', 'None', 'None', 'None', 'None', '713', 'None',
      'None', 'None', '995', 'None', '1045', '760', 'None', 'None', 'None', 'None', '799', 'None', 'None', 'None', 'None', 'None', 'None', '600', 'None', 'None',
      '700', 'None', 'None', 'None', 'None', 'None', '1200', '5500', '615', '1990', 'None', '2595', '700', '3000', '3101', '1028', 'None', 'None', '2199', 'None',
      '2800', 'None', '2144', '984', '3330', '2038', '1846', 'None', '646', 'None', 'None', '1650', 'None', '1078', '2792', '4109', 'None', 'None', '1840', '1752',
      '1116', '1850', 'None', 'None', '1114', 'None', '2208', 'None', 'None', '2913', 'None', '1154', '2169', 'None', 'None', 'None', '1029', '3787', '1800', 'None',
      '3300', 'None', '1374', '2100', 'None', 'None', '4118', 'None', 'None', 'None', 'None', 'None', 'None', '3258', '1237', 'None', '2195', '3350', 'None', '1450',
      '2265', '913', '1270', '1000', '784', '1300', '2471', '1064', 'None', '1617', '3496', '650', '1550', 'None', 'None', '1161', 'None', '1520', 'None', 'None',
      '4268', 'None', '2285', '1107', '1630', '808', 'None', '1460', '1154', '750', '1154', '1300', '1880', '29952', 'None', '3242', '861', 'None', 'None', 'None',
      'None', '1100', '1946', '1078', 'None', '961', '2700', '3261', 'None', 'None', 'None', 'None', '2901', '1600', '2948', 'None', 'None', '2147', '660', '1285',
      'None', 'None', '2352', '1626', '2463', '500', 'None', 'None', '545', '1154', 'None', '1508', '1555', '3217', '1370', 'None', '1161', 'None', 'None', '1161',
      'None', 'None', '9000', 'None', '549', 'None', 'None', '1217', 'None', '5957', '1372', '6000', '1749', 'None', 'None', '762', 'None', '545', '2666', '2253',
      '682', '907', '2152', '1639', 'None', 'None', '1930', '1116', '1000', '1300', '3350', '966', '2351', '1848', '1070', 'None', 'None', 'None', '1260', '1229',
      '856', '1445', 'None', '1740', 'None', 'None', '1956', '1116', 'None', 'None', 'None', '1994', '1614', '1154', 'None', 'None', '4909', 'None', '1886', '1776',
      'None', 'None', '861', '2364', '1446', 'None', 'None', 'None', 'None', '500', '2223', '500', '1687', '762', '450', '1154', 'None', '2076', '1013', '1237',
      '1161', '1734', '1146', '2000', '1080', '2703', '1542', '1876', '1803', 'None', '2756', 'None', '3878', 'None', 'None', '964', 'None', '2733', '2200', '500',
      'None', '2575', 'None', '2024', '1779', '1665', '1665', 'None', '1278', 'None', '1015', 'None', 'None', '1950', '1506', '1402', 'None', '1018', '1055', '5957',
      'None', 'None', '1838', '1886', '1714', '808', '1994', '2300', 'None', '3000', '2368', 'None', '2316', 'None', '1116', '3780', '1000', 'None', 'None', '500',
      '1154', '2097', 'None', 'None', 'None', '2153', 'None', '2165', 'None', 'None', '689', 'None', 'None', '3900', 'None', '3100', '2296', '1154', '1500', '3516',
      '1423', '3000', 'None', '961', 'None', 'None', '3744', '1510', '1770', '1900', '1800', 'None', '1158', '3340', '1044', '853', '1709', 'None', '1641', '1475',
      '1116', '2374', '3345', '1094', '1154', 'None', 'None', '1380', 'None', '1288', '1260', '1549', 'None', '1273', 'None', '1925', '2500', '950', 'None', 'None',
      '1975', '1005', '700', '920', '2733', '2162', 'None', '3395', 'None', 'None', '3300', '1799', '2845', '1466', '1660', 'None', 'None', '1736', '545', '2925',
      'None', 'None', 'None', '2121', 'None', '1506', 'None', 'None', 'None', '1666', 'None', '8222', 'None', 'None', '1264', '1250', '1384', 'None', 'None', '2942',
      '2200', 'None', '700', '4633', 'None', '3078', 'None', '2036', '1450', '2366', '3896', 'None', '1609', 'None', '900', '545', '1650', '2123', '2230', '2193',
      'None', '1413', '2664', 'None', '5771', '1632', '1154', 'None', '1189', 'None', '527', '1519', '920', 'None', '1583', 'None', 'None', 'None', '2000', '2560',
      '976', '913', 'None', 'None', '1436', 'None', '488', '1071', '1398', '545', '1501', '545', '2309', 'None', '1500', 'None', 'None', '1639', '2288', '2193',
      'None', '6394', 'None', 'None', '3696', '545', 'None', 'None', '1300', 'None', '749', '1887', '1772', '1766', 'None', '1284', '3430', '1894', '2824', '2418',
      '1922', 'None', '2133', '785', '1411', 'None', 'None', '1154', '2063', 'None', 'None', '1154', 'None', '2400', 'None', '1687', '1947', '1294', 'None', '1311',
      'None', 'None', '2487', '2629', '650', '1146', 'None', 'None', '1926', 'None', '1387', 'None', 'None', 'None', '1750', '4687', '1654', '1730', '1786', 'None',
      'None', 'None', 'None', '2708', '4220', '6000', 'None', 'None', 'None', '1200', 'None', '1500', '2440', '1402', 'None', '579', '1494', '1938', '941', '2250',
      '2038', '1786', 'None', 'None', '1765', '1260', '1535', '1383', 'None', 'None', '2597', 'None', '629', '6912', 'None', '2643', 'None', 'None', 'None', 'None',
      '2230', 'None', '897', '6000', 'None', 'None', '1230', '540', '1806', '1996', 'None', '500', 'None', '1562', '753', 'None', '798', 'None', '961', 'None',
      '2400', '3018', '1325', '1149', 'None', 'None', '1154', '1154', '1000', '1550', 'None', 'None', '1505', '3800', '2577', 'None', 'None', 'None', '1700', '2100',
      '2160', '1520', '1229', 'None', '625', '2470', 'None', '500', 'None', '10129', '2217', '2107', '1928', 'None', 'None', 'None', '2590', 'None', '1400', 'None',
      '1959', '1294', '4000', '3892', 'None', '2040', '3600', '1220', 'None', '1400', '627', 'None', '1764', '487', '2132', 'None', '1314', '500', '500', '2947',
      '2190', '3265', '1100', 'None', 'None', '990', 'None', '1649', 'None', '737', 'None', 'None', '1388', 'None', 'None', '1900', '1088', '1890', '917', '1166',
      '1154', '2174', '1851', '1708', '2888', '1622', '2914', '2159', '765', '2181', 'None', '2147', '1154', 'None', 'None', '2094'
      ]
      to_pop = [845, 1504, 1554, 1882, 1891, 1926, 1943] ## This had to be added due to inconsistencies in the datasets when I transfered over all the data. I'm not sure why it happened but 7 entries were unable to be processed using the code in this ipynb file to open the JSON file.
      for pop in to_pop:
          sqfootage.pop(pop)
      self.df.insert(len(self.df.iloc[0]), "square_footage", sqfootage)


    def handle_missing_values(self):
        """
        Handles missing values in the DataFrame.
        """

        # Drop rows with missing 'listPrice'
        self.df.dropna(subset=['listPrice'], inplace=True)

        # Convert relevant columns to numeric (replace non-numeric with NaN)
        numerical_features = ['yearBuilt', 'latitude', 'longitude', 'numBedrooms', 'numBathrooms', 'lotSize', 'parking', 'square_footage', 'garage']
        for col in numerical_features:
            self.df[col] = pd.to_numeric(self.df[col], errors='coerce')

        scaler = MinMaxScaler()
        numerical_features = ['yearBuilt', 'latitude', 'longitude', 'numBedrooms', 'numBathrooms', 'lotSize', 'parking', 'square_footage', 'garage']
        self.df[numerical_features] = scaler.fit_transform(self.df[numerical_features])

        # Impute missing year built using KNNImputer
        imputer = KNNImputer(n_neighbors=5)
        self.df['yearBuilt'] = imputer.fit_transform(self.df[['yearBuilt', 'propertyType_Single_Family', 'propertyType_TOWNHOUSE', 'latitude', 'longitude']])

        # Impute missing lot size using KNNImputer
        imputer = KNNImputer(n_neighbors=5)
        self.df['lotSize'] = imputer.fit_transform(self.df[['lotSize', 'numBedrooms', 'numBathrooms', 'propertyType_Single_Family', 'propertyType_TOWNHOUSE', 'parking']])

        # Impute missing square footage using KNNImputer
        imputer = KNNImputer(n_neighbors=5)
        self.df['square_footage'] = imputer.fit_transform(self.df[['square_footage', 'numBedrooms', 'numBathrooms', 'propertyType_Single_Family', 'propertyType_TOWNHOUSE', 'parking']])

        # Impute missing garage using KNNImputer
        imputer = KNNImputer(n_neighbors=5)
        self.df['garage'] = imputer.fit_transform(self.df[['garage', 'numBedrooms', 'numBathrooms', 'propertyType_Single_Family', 'propertyType_TOWNHOUSE', 'parking']])

        # Drop remaining rows with any NaN values in specified columns
        columns_to_check = ['garage', 'parking', 'numBedrooms', 'numBathrooms', 'square_footage']
        self.df.dropna(subset=columns_to_check, inplace=True)

    def one_hot_encode_categorical_features(self):
        """
        Performs one-hot encoding on selected categorical features.
        """
        categorical_features = ['propertyType', 'Amenities_Utilities_Heating_Cooling_Cooling', 'Amenities_Utilities_Heating_Cooling_Heat_Source']
        self.df = pd.get_dummies(self.df, columns=categorical_features, drop_first=True) * 1

        # Fix column names (because of values and their one-hot encoding)
        self.df.columns = self.df.columns.str.replace(r' +', '_', regex=True)

class HousingModelTrainer:
    """
    Class to train and evaluate different regression models for housing price prediction.
    """

    def __init__(self, df, test_size=0.2, random_state=42):
        """
        Initializes the HousingModelTrainer with the processed DataFrame, test size, and random state.

        Parameters:
        -----------
        df : pandas.DataFrame
            The processed DataFrame containing housing data.
        test_size : float, optional
            The proportion of the data to be used for testing (default is 0.2).
        random_state : int, optional
            The seed for random number generation (default is 42).
        """
        self.df = df
        self.test_size = test_size
        self.random_state = random_state
        self.X_train, self.X_test, self.y_train, self.y_test = self.split_data()
        self.models = self.define_models()
        self.param_grids = self.define_param_grids()
        self.best_models = self.train_models()

    def split_data(self):
        """
        Splits the data into training and testing sets.

        Returns:
        --------
        tuple
            A tuple containing X_train, X_test, y_train, y_test.
        """
        X = self.df.drop('listPrice', axis=1)
        y = self.df['listPrice']
        return train_test_split(X, y, test_size=self.test_size, random_state=self.random_state)

    def define_models(self):
        """
        Defines the regression models to be trained.

        Returns:
        --------
        dict
            A dictionary containing the models with their names as keys.
        """
        return {
            'Linear Regression': LinearRegression(),
            'Ridge': Ridge(),
            'Lasso': Lasso(max_iter=5000),
            'ElasticNet': ElasticNet(max_iter=5000),
            'Decision Tree': DecisionTreeRegressor(),
            'Random Forest': RandomForestRegressor(),
            'Gradient Boosting': GradientBoostingRegressor()
        }

    def define_param_grids(self):
        """
        Defines the hyperparameter grids for grid search.

        Returns:
        --------
        dict
            A dictionary containing the parameter grids for each model.
        """
        return {
            'Ridge': {'alpha': np.logspace(-3, 3, 10)},
            'Lasso': {'alpha': np.logspace(-3, 3, 10)},
            'ElasticNet': {'alpha': np.logspace(-5, 0, 10), 'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]},
            'Decision Tree': {'max_depth': [5, 10, 20, 35, 50], 'min_samples_split': [2, 5, 10, 25, 50]},
            'Random Forest': {'n_estimators': [100, 200, 500], 'max_depth': [5, 10, 20, 35, 50]},
            'Gradient Boosting': {'n_estimators': [100, 200, 500], 'learning_rate': [0.01, 0.1, 1]}
        }

    def train_models(self):
        """
        Trains the models using GridSearchCV for hyperparameter tuning.

        Returns:
        --------
        dict
            A dictionary containing the best trained models with their names as keys.
        """
        best_models = {}
        for name, model in self.models.items():
            if name in self.param_grids:
                grid_search = GridSearchCV(model, self.param_grids[name], cv=5, scoring='neg_mean_squared_error')
                grid_search.fit(self.X_train, self.y_train)
                best_models[name] = grid_search.best_estimator_
                print(f'Best parameters for {name}: {grid_search.best_params_}')
            else:
                model.fit(self.X_train, self.y_train)
                best_models[name] = model

        ensemble = VotingRegressor(estimators=list(best_models.items()))
        ensemble.fit(self.X_train, self.y_train)
        best_models['ensemble'] = ensemble
        print('\n***************')

        return best_models

    def evaluate_models(self):
        """
        Evaluates the trained models using various regression metrics.
        """
        for name, model in self.best_models.items():
            y_pred = model.predict(self.X_test)
            print(f'Model: {name}')
            if name == 'Lasso' or name == 'Ridge' or name == 'ElasticNet':
              coefficients = model.coef_
              feature_names = self.X_train.columns
              for i in range(len(coefficients)):
                  print(f'{feature_names[i]}: {coefficients[i]}')
              # print(f'Model: {model.coef_}')
              print(f'Features In: {model.n_features_in_}')
              print(f'Intercept: {model.intercept_}')
            # elif name == 'Decision Tree':
            print(f'Params: {model.get_params()}')
            print(f'MSE: {mean_squared_error(self.y_test, y_pred)}')
            print(f'R-squared: {r2_score(self.y_test, y_pred)}')
            print(f'MAE: {mean_absolute_error(self.y_test, y_pred)}')
            print(f'RMSE: {np.sqrt(mean_squared_error(self.y_test, y_pred))}')
            print(f'%Error: { abs(sum(self.y_test) - sum(y_pred))/sum(self.y_test) * 100 }%')
            #print('---')
            print('\n***************\n\n')



# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Colab Notebooks/3253/ontario_housing_cleaned.json'

data_processor = HousingDataProcessor(file_path)
data_processor.clean_data()

model_trainer = HousingModelTrainer(data_processor.df)
model_trainer.evaluate_models()

# Explanation of parameters passed to models:

# - alpha (Ridge, Lasso, ElasticNet): Regularization parameter. Controls the strength of regularization.
# - l1_ratio (ElasticNet): The mixing parameter between L1 and L2 penalty.
# - max_depth (Decision Tree, Random Forest): The maximum depth of the tree.
# - min_samples_split (Decision Tree): The minimum number of samples required to split an internal node.
# - n_estimators (Random Forest, Gradient Boosting): The number of trees in the forest.
# - learning_rate (Gradient Boosting): Shrinks the contribution of each tree.

# Explanation of evaluation metrics:

# - MSE (Mean Squared Error): Measures the average squared difference between the predicted values and the actual values.
# - R-squared: Indicates the proportion of the variance in the dependent variable that is predictable from the independent variables.
# - MAE (Mean Absolute Error): Measures the average absolute difference between the predicted values and the actual values.
# - RMSE (Root Mean Squared Error): The square root of MSE, provides an interpretable metric in the same units as the target variable.
# - %Error: Indicates the percentage error in the total prediction of the model on the test set.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of columns with missing values: 35 

cityCode 13.600000000000001 % missing values
closePrice 100.0 % missing values
closeDate 100.0 % missing values
listDate 38.5 % missing values
listingCoAgent 0.1 % missing values
listingAgent 0.1 % missing values
lotSizeRaw 95.19999999999999 % missing values
sqftTotalRaw 3.8 % missing values
openHouses 95.0 % missing values
parking 0.5 % missing values
yearBuiltRaw 87.4 % missing values
priceChanged 97.2 % missing values
dateHidden 100.0 % missing values
neighborhoodNGeoId 7.7 % missing values
soldDate 100.0 % missing values
propertyDateHidden 100.0 % missing values
priceChangeAmount 97.39999999999999 % missing values
thumbnail 1.7999999999999998 % missing values
photoCount1 1.7999999999999998 % missing values
garage 49.5 % missing values
virtualTourLink 74.3 % missing values
fsa 0.0 % missing values
petiteImagePath